# Assignment Requirements

Assignment Requirements:
Please set β = 0.8, and initial PageRank value = 1/N.
Show the top 10 vertices sorted by rank (if ranks are equal, sorted by ID in
ascending order), after 20 iterations.
Part1 Code(80%)
Please make sure that your file has the same name as PageRank.
(PageRank.java or PageRank.ipynb)
Part2 Report(20%)
Java :
1. Report.pdf (Explain how do you design your mapper and reducer)
2. Outputfile.txt (write your result of input.txt to this file)
Python:
1. Report.pdf or markdown in .ipynb file (Explain how do you design
your mapper and reducer.)
2. Outputfile.txt (write your result of input.txt to this file)
Please pack the above files into a zip file. Name it as
“MDA_HW2_studentID.zip”
Should notice:
- How to get ?
- (-5) Wrong output format
- (-5) Round-off error
- (-10) Cannot output the result of input-test.txt
N

## Configuation

In [2]:
import os
import sys
from pyspark.context import SparkContext,SparkConf
from pyspark.rdd import RDD
!pip install psutil
import psutil
#若沒有下面這段
#會有Exception: Java gateway process exited before sending its port number
#Config
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jre1.8.0_301'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


sc = SparkContext(conf=SparkConf().setAppName("PageRank").setMaster("local"))
print(f'sc:{sc}')
spark = SparkSession(sc)


sc:<SparkContext master=local appName=PageRank>


## Import dataset 
1. mini_test data: input_test.txt

In [3]:
#create a little txt file named input_test.txt to test 
p_path = 'C:\\Users\\LeoShr\\p_space\\NTHU\\MDA\\CH2_PageRank_Pyspark\\input_test.txt'
rdd1  =sc.textFile(p_path)

# The code below is the designing procedure, which can be skipped

if you want to see the final result, kindly ctrl+f for the 

## formating

 transform ['1\t2'] into ['1','2']

In [ ]:
rdd2 = rdd1.map(lambda x:  (x.split("\t")[0], x.split("\t")[1])).cache()

print(rdd2.take(14))


### Setting Start node as the key of pair RDD

In [ ]:
pages = rdd2.groupByKey().map(lambda x : (x[0], list(x[1])))
#rank = rdd2.map(lambda x: (x[0],x[1]))#這也可
pages.collect()


## The code below should be looped k times, k is the question requirement.

# Build the key of the  current rank_value for the nodes with outlink 


### What is N???

1. N is the total amount of the unique pages, links or websites we have.


### How to get N???

1. it's a directed graph. it takes us O(v) to visit all the nodes from a specific point.  Even if we stored data as adjacency list,(as we just built), it still requires O(E+V) time to travel through the whole graph. And the most important of all, I don't know how to compute this idea via mapreduce

2. we just count the element for the input data and pick the distinct one out of them.

In [ ]:
num_a_spot = rdd2.map(lambda x : x[0])
num_b_spot = rdd2.map(lambda x : x[1])
node_list = sorted(num_a_spot.union(num_b_spot).distinct().collect())
N = len(node_list) #

####  Build the key of the current rank_value for the nodes with outlink

In [ ]:
links = pages.map(lambda x : (x[0], 1/N))
links.collect()

### Scatter

### Define a function scatter

this function can distribution the rank value depends our outlink numbers

In [5]:
def scatter(x):
    temp = []
    len_outlink_num = len(x[1][0]) #看有多少outlink
    for y in x[1][0]:
        temp.append(tuple((y,x[1][1]/len_outlink_num)))
    return temp

In [ ]:
rank = pages.join(links).flatMap(scatter)
rank.collect()

In [ ]:
#reduce
links = rank.reduceByKey(lambda x,y: x+y)

links.collect()

### revised check

In [ ]:
#revised check
links_revised = links.mapValues(lambda x: 0.8*x+ 0.2*(1/N))
links_revised.collect()

###  Renormaliz to avoid spider  trap

Algo: 
1. sum all the value of the pair RDD, and named it as s
2. count (1-S)/N

In [ ]:
print(links_revised.collect())
temp = links_revised.map(lambda x: x[1])
temp.collect()

In [ ]:
s = sum(temp.collect())
#print(f's:{s}')
normalized_residual = (1-s)/N
#print(f'normalized_residual: {normalized_residual}')

In [ ]:
#renormalize check
links = links_revised.mapValues(lambda x: x+normalized_residual)
links.collect()

## Algo Design


### 10個迭代的版本

### 實際測資

### Revised PageRank Algo Design

##### 1. input: 
iterating_time:指定迭代次數
path: input檔案的路徑
saved_named: 儲存檔案的名稱

#####  2. formatting
sc.textFile(path).map(lambda x:  (x.split("\t")[0], x.split("\t")[1])).cache()

1. read the txt file and turn the original format (transform ['1\t2']) into (['1','2'])
2. Setting the start node as format, and the end node as the element of a list, which sites at the place of j, eg,  [('0',('1','2','3','5')),('1',('2', '3', '1'))


##### 3. Building second  RDD for recording the rank value

Build a RDD having the same key as the previous RDD we converted, like [('1', page_value)]

1. num_a_spot: we extract the key as a sigle RDD called num_a_spot

2. num_b_spot: we extract the value as a sigle RDD called num_b_spot

3. discinct them and ->collect() -> sort them out  what exactly how many ending node this start node has.

4. get the result of (3) into the len() function to understand the total amount of node number we need to scatter the weights (RankValue/N)

5. links = pages.map(lambda x : (x[0], 1/N)) would return the RDD which looks like [(key, the average RankValue)]

##### 4. Scatter the PageRank 
1. iterating as the input 'iterating_time'
2. Scatter the value vias mapping
3. reduce: sum the value of the pair RDD by key. Therefore, we can get the total amount of updated RankValue with a unique key
##### 5. Revised check
1. depends on the formula: {rj =Σi→j  [β(ri/di)+ (1 − β)s/N]}, we should consider the effect of β and (1 − β)s/N,  β = 0.8, s = the difference between 1 and the total sum of current nodes' rank value

2. we use mapping function to parallel computing the revised rank values

3. sorting 1. sorting by rankvalue descending , therefore, the rdds with higher ranking are being sorted frontier  

4. Use links_revised.mapValues(lambda x: (x+normalized_residual)) to make up the difference and avoid spider trap's effect

##### 6. Round off check
1. the assignment requires ound to the third non-zero digit after decimal point

2. After the observation, we set the round function to truncate at the sixth digit after the point can match the requirement


3. We only conduct this section of code in the last round to avoid the truncation error ('iterating_time-1' due to python start from 0, hence, 19th round should bbe the last round instead of the round 20)


##### 7. Auto_stored

1. we can auto store the result as our requirement. ps, the input parameter includes the stored output name we want.


#### Return the RDD result


In [9]:
#別人的out-link 分給我們的value
#flatMap(f), f是在做根據 X[1][1]的長度，也就是自己outlink的數量進行分配給其他node
def scatter(x):
    temp = []
    len_outlink_num = len(x[1][0]) #看有多少outlink
    for y in x[1][0]:
        temp.append(tuple((y,x[1][1]/len_outlink_num)))
    return temp

In [8]:
##########this is the for loop function we defined
p_path = 'C:\\Users\\LeoShr\\p_space\\NTHU\\MDA\\CH2_PageRank_Pyspark\\input.txt'
iterating_time = 20
saved_named = 'testing4_officail_datatesting.txt'
def Re_PageRank(iterating_time,path,saved_named):
    ######  2. formatting
    rdd2 = sc.textFile(path).map(lambda x:  (x.split("\t")[0], x.split("\t")[1])).cache()
    pages = rdd2.groupByKey().map(lambda x : (x[0], list(x[1])))    
    
    ###### 3. Building second  RDD for recording the rank value   
    num_a_spot = rdd2.map(lambda x : x[0])
    num_b_spot = rdd2.map(lambda x : x[1])
    node_list = sorted(num_a_spot.union(num_b_spot).distinct().collect())
    N = len(node_list) #the total node number
    links = pages.map(lambda x : (x[0], 1/N))
    ##### 4. Scatter the PageRank 
    for time in range(iterating_time):
      
        #join two RDD as the format (key, ([outlink_nodes],rank_value))
        #scatter: map
        rank = pages.join(links).flatMap(scatter)
        #reduce: sum the value of the pair RDD by key
        links = rank.reduceByKey(lambda x,y: x+y)
        
        
        ###### 5. Revised check
        #revised: depends on the formula: {rj =Σi→j  β(ri/di)+ (1 − β) s/N}
        #Revised check preparation:
        links_revised = links.mapValues(lambda x: 0.8*x+ 0.2*(1/N))
        temp = links_revised.map(lambda x: x[1])
        s = sum(temp.collect())
        normalized_residual = (1-s)/N
     
        #sorting 1. sorting by rank decending
        #sortbyvalue desending , therefore, the rdds with higher ranking are sorted frontier  
        links_revised = links.sortBy(lambda a: -a[1])
        links = links_revised.mapValues(lambda x: (x+normalized_residual))
        print(f'i am rounded! And it is {time} round')

        if time == iterating_time-1: 
        #擔心每次都截尾會造成大迭代次數時的誤差，因此最後一次迭代時再四捨五入到非零整數第三位
            result = links_revised.mapValues(lambda x: round(x,6))
            print('i am rounded! And it is the final round')
## 7. Auto_stored 
   
    fp = open(saved_named, 'w')
    for element in result.collect():
        jj = str(element[0])+'    '+str(element[1])
        #print(jj)
        fp.write(jj+'\n')

    fp.close()   
    return result#.collect()
    #result.saveAsTextFile('./Page_rank_result')    會有error
        

ans = Re_PageRank(iterating_time,p_path,saved_named).collect()
    
    
ans

i am rounded! And it is 0 round
i am rounded! And it is 1 round
i am rounded! And it is 2 round
i am rounded! And it is 3 round
i am rounded! And it is 4 round
i am rounded! And it is 5 round
i am rounded! And it is 6 round
i am rounded! And it is 7 round
i am rounded! And it is 8 round
i am rounded! And it is 9 round
i am rounded! And it is 10 round
i am rounded! And it is 11 round
i am rounded! And it is 12 round
i am rounded! And it is 13 round
i am rounded! And it is 14 round
i am rounded! And it is 15 round
i am rounded! And it is 16 round
i am rounded! And it is 17 round
i am rounded! And it is 18 round
i am rounded! And it is 19 round
i am rounded! And it is the final round


[('4240', 0.000652),
 ('10861', 0.000635),
 ('9526', 0.00053),
 ('6899', 0.000513),
 ('2118', 0.000498),
 ('3186', 0.000489),
 ('1311', 0.000488),
 ('3419', 0.000478),
 ('7233', 0.000468),
 ('353', 0.000452),
 ('3541', 0.000452),
 ('367', 0.000449),
 ('5734', 0.00042),
 ('5819', 0.000418),
 ('10749', 0.000409),
 ('6454', 0.000398),
 ('4366', 0.000397),
 ('1558', 0.000393),
 ('251', 0.000388),
 ('2746', 0.000384),
 ('7847', 0.00038),
 ('6514', 0.000379),
 ('5169', 0.000378),
 ('2994', 0.000377),
 ('4709', 0.000375),
 ('8123', 0.000372),
 ('4189', 0.000371),
 ('5055', 0.000363),
 ('2653', 0.000362),
 ('3185', 0.000361),
 ('2859', 0.000359),
 ('6340', 0.000354),
 ('10652', 0.000351),
 ('10844', 0.000349),
 ('6426', 0.000347),
 ('687', 0.000342),
 ('7880', 0.000341),
 ('5825', 0.000339),
 ('7829', 0.000338),
 ('2571', 0.000327),
 ('2355', 0.000326),
 ('7922', 0.000325),
 ('4525', 0.000321),
 ('5985', 0.00032),
 ('2773', 0.000317),
 ('8549', 0.000316),
 ('4193', 0.000314),
 ('2344', 0.00030